## Install Libraries

In [6]:
%pip install torch torchvision
%pip install pillow
%pip install scikit-learn
%pip install opencv-python


  Using cached torch-2.3.1-cp312-cp312-win_amd64.whl.metadata (26 kB)
  Using cached torchvision-0.18.1-cp312-cp312-win_amd64.whl.metadata (6.6 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.12.1-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.13.0-py3-none-win_amd64.whl.metadata (1.1 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.3.1-cp312-cp312-win_amd64.whl (159.7 MB)
Using cached torchvision-0.18.1-cp312-cp312-win_amd64.whl (1.2 MB)
   ---------------------------------------- 0.0/228.5 MB ? eta -:--:--
   ---------------------------------------- 0


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   -- ------------------------------------ 14.5/228.5 MB 246.3 kB/s eta 0:14:30
   -- ------------------------------------ 14.5/228.5 MB 246.3 kB/s eta 0:14:30
   -- ------------------------------------ 14.5/228.5 MB 246.3 kB/s eta 0:14:30
   -- ------------------------------------ 14.5/228.5 MB 246.3 kB/s eta 0:14:30
   -- ------------------------------------ 14.6/228.5 MB 247.7 kB/s eta 0:14:24
   -- ------------------------------------ 14.7/228.5 MB 252.9 kB/s eta 0:14:06
   -- ------------------------------------ 14.7/228.5 MB 252.9 kB/s eta 0:14:06
   -- ------------------------------------ 14.7/228.5 MB 252.9 kB/s eta 0:14:06
   -- ------------------------------------ 14.7/228.5 MB 252.9 kB/s eta 0:14:06
   -- ------------------------------------ 14.7/228.5 MB 252.9 kB/s eta 0:14:06
   -- ------------------------------------ 14.7/228.5 MB 252.9 kB/s eta 0:14:06
   -- ------------------------------------ 14.7/228.5 MB 252.9 kB/s eta 0:14:06
   -- ----------------------------------


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Loading and splitting data

In [6]:
import os
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
from concurrent.futures import ThreadPoolExecutor

# Define the path to the images folder
image_folder = 'extracted_images'

# Get a list of all image files in the folder
image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.png', '.jpg', '.jpeg'))]

# Limit the number of images to 50,000
start = 0
end = len(image_files)-1
step = 3
image_files = image_files[start:end:step]

# Split the data into training, testing, and validation sets
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)
train_files, val_files = train_test_split(train_files, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Function to extract the label from the filename
def extract_label_from_filename(filename):
    # Remove the extension from the filename
    return os.path.splitext(filename)[0]

# Function to load a single image and its label
def load_image_and_label(file):
    # Load the image
    image = Image.open(file).convert('L')  # Convert to grayscale
    image = np.array(image)

    # Extract the label from the filename
    label = extract_label_from_filename(os.path.basename(file))
    return image, label

# Function to load images and extract labels using parallel processing
def load_images_and_labels(files):
    images = []
    labels = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        results = list(executor.map(load_image_and_label, files))
    for image, label in results:
        images.append(image)
        labels.append(label)
    
    return np.array(images), np.array(labels)

# Load images and labels for training, validation, and testing sets
train_images, train_labels = load_images_and_labels(train_files)
val_images, val_labels = load_images_and_labels(val_files)
test_images, test_labels = load_images_and_labels(test_files)

# Print the number of files in each set
print(f"Total images: {len(image_files)}")
print(f"Training set: {len(train_files)}")
print(f"Validation set: {len(val_files)}")
print(f"Testing set: {len(test_files)}")

# Optionally, print the first few labels to verify
print("Sample labels from training set:", train_labels[:5])
print("Sample labels from validation set:", val_labels[:5])
print("Sample labels from testing set:", test_labels[:5])


Total images: 86893
Training set: 52135
Validation set: 17379
Testing set: 17379
Sample labels from training set: ['05_32_20_28_30_24_dilation' '05_32_24_20_36_02_shear'
 '05_32_02_20_32_34_dilation' '05_32_30_32_24_32_none'
 '05_32_34_20_36_11_erosion']
Sample labels from validation set: ['05_32_24_28_31_24_dilation' '05_32_28_34_20_02_dilation'
 '05_32_18_27_34_01_shear' '05_32_36_24_27_33_rotation'
 '05_32_07_28_07_02_erosion']
Sample labels from testing set: ['05_32_32_36_32_37_dilation' '05_32_27_18_31_37_rotation'
 '05_32_34_27_02_14_dilation' '05_32_24_30_07_01_rotation'
 '05_32_20_02_26_20_dilation']


# Preprocess Data

In [7]:
import tensorflow as tf
import numpy as np
import cv2

def load_image(image_path):
    image = cv2.imread(image_path)  # Load image in RGB format
    image = cv2.resize(image, (128, 32))  # Resize image to (128, 32)
    image = image.astype(np.float32) / 255.0  # Normalize pixel values
    return image

def preprocess_data(image_files, labels):
    images = [load_image(file) for file in image_files]
    labels = [label.split('_')[1:-1] for label in labels]  # Extract character indices
    labels = [[int(char) for char in label] for label in labels]  # Convert to integers
    labels = np.array(labels)
    return np.array(images), labels

# Preprocess the data
train_images, train_labels = preprocess_data(train_files, train_labels)
val_images, val_labels = preprocess_data(val_files, val_labels)
test_images, test_labels = preprocess_data(test_files, test_labels)

print(train_images.shape)
print(train_labels.shape)
print(val_labels.shape)


(52135, 32, 128, 3)
(52135, 5)
(17379, 5)


# Convert to Categorical Labels

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the Urdu alphabet and create an index dictionary
urdu_alphabet = ["ا", "ب", "پ", "ت", "ٹ", "ث", "ج", "چ", "ح", "خ", "د", "ڈ", "ذ", "ر", "ڑ", "ز", "ژ", "س", "ش", "ص", "ض", "ط", "ظ", "ع", "غ", "ف", "ق", "ک", "گ", "ل", "م", "ن", "و", "ہ", "ء", "ی", "ے"]
urdu_index_dict = {char: idx + 1 for idx, char in enumerate(urdu_alphabet)}

# Function to convert label indices to categorical format using the index dictionary
def labels_to_categorical(labels, index_dict, num_classes):
    categorical_labels = []
    for label in labels:
        categorical_label = []
        for index in label:
            if index < 1 or index > len(index_dict):
                raise ValueError(f"Label index '{index}' is out of range or not found in urdu_alphabet.")
            categorical_label.append(tf.keras.utils.to_categorical(index - 1, num_classes=num_classes))  # Adjust index to start from 0
        categorical_labels.append(categorical_label)
    return np.array(categorical_labels)

# Convert labels to categorical format
train_labels_categorical = labels_to_categorical(train_labels, urdu_index_dict, len(urdu_alphabet))
val_labels_categorical = labels_to_categorical(val_labels, urdu_index_dict, len(urdu_alphabet))

# Print the length of converted labels for verification
print(len(train_labels_categorical))
print(len(val_labels_categorical))

# Optionally, you may print the index dictionary for verification
print(urdu_index_dict)


52135
17379
{'ا': 1, 'ب': 2, 'پ': 3, 'ت': 4, 'ٹ': 5, 'ث': 6, 'ج': 7, 'چ': 8, 'ح': 9, 'خ': 10, 'د': 11, 'ڈ': 12, 'ذ': 13, 'ر': 14, 'ڑ': 15, 'ز': 16, 'ژ': 17, 'س': 18, 'ش': 19, 'ص': 20, 'ض': 21, 'ط': 22, 'ظ': 23, 'ع': 24, 'غ': 25, 'ف': 26, 'ق': 27, 'ک': 28, 'گ': 29, 'ل': 30, 'م': 31, 'ن': 32, 'و': 33, 'ہ': 34, 'ء': 35, 'ی': 36, 'ے': 37}


# Ensure Correct Data Shapes

In [9]:
import numpy as np

# Verify the shapes of the data
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels_categorical.shape)

# Ensure the input data is in the correct shape
train_images = np.array(train_images)
train_labels_categorical = np.array(train_labels_categorical)

# Ensure that the number of samples match
assert train_images.shape[0] == train_labels_categorical.shape[0], "Number of samples in train_images and train_labels_categorical do not match!"

# Check the shapes of validation data
print("Validation images shape:", val_images.shape)
print("Validation labels shape:", val_labels_categorical.shape)

# Ensure the validation data is in the correct shape
val_images = np.array(val_images)
val_labels_categorical = np.array(val_labels_categorical)

# Ensure that the number of samples match
assert val_images.shape[0] == val_labels_categorical.shape[0], "Number of samples in val_images and val_labels_categorical do not match!"

# Print shapes to verify
print("Corrected train images shape:", train_images.shape)
print("Corrected train labels shape:", train_labels_categorical.shape)
print("Corrected validation images shape:", val_images.shape)
print("Corrected validation labels shape:", val_labels_categorical.shape)


Train images shape: (52135, 32, 128, 3)
Train labels shape: (52135, 5, 37)
Validation images shape: (17379, 32, 128, 3)
Validation labels shape: (17379, 5, 37)
Corrected train images shape: (52135, 32, 128, 3)
Corrected train labels shape: (52135, 5, 37)
Corrected validation images shape: (17379, 32, 128, 3)
Corrected validation labels shape: (17379, 5, 37)


In [10]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Bidirectional, LSTM, TimeDistributed, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Define the number of classes
num_classes = 37  # 36 Urdu characters + 1 blank for CTC

# Load VGG16 without the top layer and set it to be untrainable
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 128, 3))
for layer in vgg16.layers:
    layer.trainable = False

# Create a new model with VGG16 as a feature extractor
input_layer = Input(shape=(32, 128, 3))
x = vgg16(input_layer)

# Flatten the output from VGG16
x = Flatten()(x)

# Reshape to (time_steps, features) for LSTM
x = Dense(1280)(x)
x = Reshape((5, 256))(x)

# Add Bidirectional LSTM
x = Bidirectional(LSTM(256, return_sequences=True))(x)

# Add a TimeDistributed Dense layer
x = TimeDistributed(Dense(128, activation='relu'))(x)
x = TimeDistributed(Dropout(0.5))(x)
output_layer = TimeDistributed(Dense(num_classes, activation='softmax'))(x)

# Create the final model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Define callbacks
checkpoint = ModelCheckpoint('best_model_vgg16_blstm.keras', monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 128, 3)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 1, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1280)           │     2,622,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 5, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 5, 512)         │     1,050,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 5, 128)         │        65,664 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 5, 128)         │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 5, 37)          │         4,773 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,458,469 (70.41 MB)

 Trainable params: 3,743,781 (14.28 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [11]:
# Train the model with the actual data
model.fit(
    train_images, train_labels_categorical,
    validation_data=(val_images, val_labels_categorical),
    epochs=50,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

Epoch 1/50
1630/1630 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step - accuracy: 0.3892 - loss: 1.8396
Epoch 1: val_loss improved from inf to 1.12176, saving model to best_model_vgg16_blstm.keras
1630/1630 ━━━━━━━━━━━━━━━━━━━━ 969s 592ms/step - accuracy: 0.3892 - loss: 1.8394 - val_accuracy: 0.6093 - val_loss: 1.1218 - learning_rate: 0.0010
Epoch 2/50
1630/1630 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step - accuracy: 0.6026 - loss: 1.1498
Epoch 2: val_loss improved from 1.12176 to 0.96248, saving model to best_model_vgg16_blstm.keras
1630/1630 ━━━━━━━━━━━━━━━━━━━━ 964s 591ms/step - accuracy: 0.6026 - loss: 1.1497 - val_accuracy: 0.6593 - val_loss: 0.9625 - learning_rate: 0.0010
Epoch 3/50
1630/1630 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step - accuracy: 0.6538 - loss: 0.9913
Epoch 3: val_loss improved from 0.96248 to 0.84530, saving model to best_model_vgg16_blstm.keras
1630/1630 ━━━━━━━━━━━━━━━━━━━━ 961s 590ms/step - accuracy: 0.6538 - loss: 0.9912 - val_accuracy: 0.7007 - val_loss: 0.8453 - learning_rate: 0.0010
Ep

KeyboardInterrupt: 

# Validation

In [12]:
# Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(val_images, val_labels_categorical)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


544/544 ━━━━━━━━━━━━━━━━━━━━ 287s 526ms/step - accuracy: 0.7828 - loss: 0.6148
Validation Loss: 0.6137086749076843
Validation Accuracy: 0.7820818424224854


# Testing

In [13]:

# Convert test labels to categorical format
test_labels_categorical = labels_to_categorical(test_labels, urdu_index_dict, num_classes)

# Ensure the input data is in the correct shape
test_images = np.array(test_images)
print("Test labels:", test_labels_categorical.shape)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels_categorical)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


Test labels: (17379, 5, 37)
544/544 ━━━━━━━━━━━━━━━━━━━━ 331s 608ms/step - accuracy: 0.7802 - loss: 0.6093
Test Loss: 0.6043021082878113
Test Accuracy: 0.7828068137168884
